In [ ]:
%cd ..

In [ ]:
# Python imports
import os
import sys

# Add the path to the parent directory to augment search for module
sys.path.append(os.getcwd())
# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if par_dir not in sys.path:
    sys.path.append(par_dir)
    
# ML imports
import torch
import matplotlib.pyplot as plt
import numpy as np

# DiVAE imports
from models.rbm.chimeraRBM import ChimeraRBM
from models.rbm.rbm import RBM
from models.samplers.pcd import PCD

from nbutils import *

# DWave imports
from dwave.system import DWaveSampler, LeapHybridSampler
import neal

In [ ]:
_NUM_NODES = 2
crbm = ChimeraRBM(n_visible=_NUM_NODES, n_hidden=_NUM_NODES)
crbm_sampler = PCD(batch_size=1024, RBM=crbm, n_gibbs_sampling_steps=100)

In [ ]:
crbm_vis, crbm_hid = crbm_sampler.block_gibbs_sampling()

In [ ]:
crbm = crbm_sampler.get_rbm()
crbm_weights, crbm_vbias, crbm_hbias = crbm.weights, crbm.visible_bias, crbm.hidden_bias

In [ ]:
crbm_energies = sample_energies(crbm, crbm_vis, crbm_hid).detach().cpu().numpy()

In [ ]:
plot_sample_energies(crbm_energies)

In [ ]:
ising_weights = crbm_weights/4.
ising_vbias = crbm_vbias/2. + torch.sum(crbm_weights, dim=0)/4.
ising_hbias = crbm_hbias/2. + torch.sum(crbm_weights, dim=1)/4.

In [ ]:
zero = torch.tensor(0., dtype=torch.float)
minus_one = torch.tensor(-1., dtype=torch.float)

ising_vis = torch.where(crbm_vis == zero, minus_one, crbm_vis)
ising_hid = torch.where(crbm_hid == zero, minus_one, crbm_hid)

In [ ]:
ising_energies = ising_energies_exp(ising_weights, ising_vbias, ising_hbias, ising_vis, ising_hid).detach().cpu().numpy()

In [ ]:
plot_sample_energies(ising_energies)